# Import dependencies

1. Descargamos las librerías necesarias para la realización del script:

In [1]:
################  TRANSFORMERS ################
!pip  install transformers --quiet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import Trainer
!pip install transformers[torch] --quiet
!pip install accelerate -U --quiet
############## OPTUNA
! pip install optuna
import optuna
from optuna import Trial
from typing import Dict, Union
import threading
! pip install optuna-dashboard
! optuna-dashboard sqlite:///db.sqlite3
################  DATASETS ################
!pip install Datasets --quiet
from datasets import DatasetDict, Dataset
# Librería para montar drive en GoogleColab y descargar el dataset que vamos a usar:
from google.colab import drive  #Montador de drive


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 1967, in _exec_single_context
    self.dialect.do_execute(
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 924, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: version_info

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 72, in _create_scoped_session
    yield session
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 1042, in _init_version_info_model
    version_info = models.VersionInfoModel.find(session)
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/models.py", line 578, in find
    version_info = session.query(cls).one_or_none()
  File "/usr/local/lib/python3.10/dist-packages/sqlalche

In [2]:
# Importamos otros modulos que necesitaremos para nuestra aproximación:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
################  MÉTRICAS ################
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Librería usada para la división del conjunto de datos en 3 conjuntos (Entrenamiento, Validación y Test)
from sklearn.model_selection import train_test_split
import gc

# 2. Importamos el dataset

In [3]:
# 1.Obtenemos de drive el dataframe con el que vamos a trabajar
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/datasets/01 Exist2021_Esp.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df.head()

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,es,Nadie te va a tratar tan bien como un hombre q...,sexist,sexual-violence
1,EXIST2021,2,twitter,es,"@lindagisela74 Que rica putita obediente, afor...",sexist,stereotyping-dominance
2,EXIST2021,3,twitter,es,@BicireporteraDF Yo lo hice a los 18 años por ...,non-sexist,non-sexist
3,EXIST2021,4,twitter,es,las cosas q sueño son indicios de que yo enrea...,non-sexist,non-sexist
4,EXIST2021,5,twitter,es,"Pero a la niña le gustó desde que lo vió, así ...",non-sexist,non-sexist


In [5]:
# 2. Cambiamos el nombre de las columnas e eliminamos aquellas que no necesitamos:
columns_to_remove = ['test_case', 'id', 'source','language','task2']
df = df.rename(columns = {"task1": "label"}).drop(columns=columns_to_remove, axis=1)
df.head()

,text,label
0,Nadie te va a tratar tan bien como un hombre q...,sexist
1,"@lindagisela74 Que rica putita obediente, afor...",sexist
2,@BicireporteraDF Yo lo hice a los 18 años por ...,non-sexist
3,las cosas q sueño son indicios de que yo enrea...,non-sexist
4,"Pero a la niña le gustó desde que lo vió, así ...",non-sexist


In [6]:
#3. Cambiamos los valores nominales de sexista y no sexista a valores numéricos
df['label'] = df['label'].replace(['non-sexist','sexist'],[0, 1])

In [7]:
#4. Eliminamos cualquier fila que haya podido quedar en blanco:
df = df.dropna()

In [8]:
df.head()

,text,label
0,Nadie te va a tratar tan bien como un hombre q...,1
1,"@lindagisela74 Que rica putita obediente, afor...",1
2,@BicireporteraDF Yo lo hice a los 18 años por ...,0
3,las cosas q sueño son indicios de que yo enrea...,0
4,"Pero a la niña le gustó desde que lo vió, así ...",0


https://www.youtube.com/watch?v=ZvsH09XGuZ0

### 2.1 - *Separamos el dataset en varios conjuntos (Train, Validación y Test)*

In [9]:
# Dividiremos el dataset en el 80% para el entrenamiento, el 10% para el proceso de validation, y un 10% para testear los resultados del modelo.
train_df, valtest_df = train_test_split(df, test_size = 0.2, random_state = 42)
val_df, test_df = train_test_split(valtest_df, test_size = 0.5, random_state = 42)
train_df.shape, val_df.shape, test_df.shape

((9063, 2), (1133, 2), (1133, 2))

In [10]:
train = Dataset.from_pandas(train_df)
validation = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(test_df)

In [11]:
#Lo convertimos en un solo dataset:

TWS = DatasetDict({
  'train': train,
  'val': validation,
  'test': test})
# Dataset.from_pandas genera una columna denominada index_0 qeu debemos eliminar:

TWS = TWS.remove_columns(["__index_level_0__"])

In [12]:
#Especificamos el modelo a adaptar a nuestro problema:
model_ckpt = 'sdadas/xlm-roberta-large-twitter'
# y descargamos su tokenizador:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
#Debemos definir una función para tokenizar el dataset y que sea gestionable por nuestro modelo:
def tokenizador(batch):
  return tokenizer( batch["text"], padding = True,  max_length=128, truncation = True, return_tensors="pt")

In [14]:
# Para poder aplicarlo al todo el diccionario utilizaremos la función map:
TWS_Encoded = TWS.map(tokenizador, batched = True,batch_size = None )

Map:   0%|          | 0/9063 [00:00<?, ? examples/s]

Map:   0%|          | 0/1133 [00:00<?, ? examples/s]

Map:   0%|          | 0/1133 [00:00<?, ? examples/s]

In [15]:
#Necesitamos que los objetos sean de tipo torch
TWS_Encoded.set_format("torch", columns = ["label","input_ids", "attention_mask"])

In [16]:
#Debemos comprobar que la ejecucion se vaya a realizar sobre cuda:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# Definiremos un diccionario para poder traduccir las etiquetas o labels que nos devuelva nuestro modelo:
Etiqueta_dato = {}
Dato_Etiqueta = {}

etiquetas = ['non-sexist','sexual-violence', 'stereotyping-dominance',
 'misogyny-non-sexual-violence' ,'ideological-inequality', 'objectification']
for dato, nombre_etiqueta in enumerate(etiquetas):
  Dato_Etiqueta[nombre_etiqueta] = str(dato)
  Etiqueta_dato[str(dato)]= nombre_etiqueta


Etiqueta_dato , Dato_Etiqueta

({'0': 'non-sexist',
  '1': 'sexual-violence',
  '2': 'stereotyping-dominance',
  '3': 'misogyny-non-sexual-violence',
  '4': 'ideological-inequality',
  '5': 'objectification'},
 {'non-sexist': '0',
  'sexual-violence': '1',
  'stereotyping-dominance': '2',
  'misogyny-non-sexual-violence': '3',
  'ideological-inequality': '4',
  'objectification': '5'})

# 3 - Análisis de Hiperparametros:

In [18]:
# 1. Definimos la función con los hyperparametros que queremos probar:
def my_hp_space(trial):
    return {
        "learning_rate" : trial.suggest_categorical('learning_rate', [1e-5, 3e-5, 2e-5]),
        "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16, 32]),
        #"per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8,12,16]),
        "num_train_epochs":trial.suggest_int("num_train_epochs", low=3, high=4)
        #"dropout_rate": trial.suggest_int("dropout_rate", low=0,1, high=0,4)
    }

In [19]:
# 2. Tenemos que definir una función de iniciado del modelo
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = 2)

In [20]:
# 3. vamos a necesitar para nuestro objeto trainer la función que calcula las métricas:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [21]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    per_device_eval_batch_size=64,
    evaluation_strategy = "epoch",
    logging_dir=None,                # directory for storing logs, set to None to disable
    logging_steps=5000,              # increase the number of steps between logs to reduce log size
    save_strategy="no"               # no saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# 3. Crear un objeto entrenador con la función model_init, argumentos de entrenamiento, conjuntos de datos de entrenamiento y prueba, y función de evaluación:
trainer = Trainer(
    args=training_args,
    train_dataset = TWS_Encoded["train"],
    eval_dataset = TWS_Encoded["val"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,

)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Ejecutamos el estudio en optuna a través de la API dEL TRAINER.
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=my_hp_space,
    n_trials= 8,
)

[I 2024-05-26 14:00:44,827] A new study created in memory with name: no-name-786d4e85-1ea7-48a9-8ac9-3a11d5e0e4cf
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.505184,0.789938,0.787198
2,No log,0.553888,0.796999,0.795401
3,No log,1.000224,0.789056,0.789104


[I 2024-05-26 14:44:43,666] Trial 0 finished with value: 1.5781595286107921 and parameters: {'learning_rate': 2e-05, 'batch_size': 32, 'num_train_epochs': 3}. Best is trial 0 with value: 1.5781595286107921.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.496564,0.778464,0.776611
2,No log,0.668628,0.805825,0.805558
3,No log,1.081685,0.796999,0.797047
4,No log,1.154479,0.803177,0.803224


[I 2024-05-26 15:44:10,700] Trial 1 finished with value: 1.6064014305674101 and parameters: {'learning_rate': 2e-05, 'batch_size': 16, 'num_train_epochs': 4}. Best is trial 1 with value: 1.6064014305674101.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2024-05-26 18:27:03,999] Trial 5 pruned. 
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.463810,0.792586,0.791179
2,No log,0.547092,0.801412,0.801367
3,No log,1.003597,0.796999,0.797048


[I 2024-05-26 19:11:28,024] Trial 6 finished with value: 1.5940466361073942 and parameters: {'learning_rate': 1e-05, 'batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 1.6064014305674101.
Trying to set batch_size in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.510856,0.754634,0.750561


[I 2024-05-26 19:26:17,492] Trial 7 pruned. 


In [24]:
preds_output = trainer.predict(TWS_Encoded["test"])
preds_output.metrics

{'test_loss': 0.5038334131240845,
 'test_accuracy': 0.7502206531332745,
 'test_f1': 0.7452363329151438,
 'test_runtime': 23.3704,
 'test_samples_per_second': 48.48,
 'test_steps_per_second': 0.77}